In [2]:
import rdflib
from pathlib import Path
import os
import re

In [3]:
in_dir = Path('../data/out/citypulse')
out_dir = Path('../data/out/citypulse-graphs')

In [11]:
at = rdflib.URIRef('http://purl.org/NET/c4dm/timeline.owl#at')
g_prefix = 'http://iot.ee.surrey.ac.uk/citypulse/datasets/traffic/traffic_oct_nov/graphs/'
rdflib_bug_rx = re.compile(r'^.*<urn:x-rdflib:>.*$')

for d1 in sorted(os.listdir(in_dir)):
    for d2 in sorted(os.listdir(in_dir / d1)):
        for f in sorted(os.listdir(in_dir / d1 / d2)):
            if f.endswith('.ttl'):
                out_path = (out_dir / d1 / d2 / (f[:-4] + '.trig'))
                if out_path.exists():
                    continue
                g = rdflib.Graph()
                g.parse(in_dir / d1 / d2 / f, format='turtle')
                g_node = rdflib.URIRef(g_prefix + f[:-4])
                time = list(g.objects(None, at))[0]
                d = rdflib.Dataset()
                for t in g.triples((None, None, None)):
                    d.add((t[0], t[1], t[2], g_node))

                d.add((g_node, at, time))
                (out_dir / d1 / d2).mkdir(parents=True, exist_ok=True)
                serialized = d.serialize(format='trig')
                # workaround for rdflib bug
                serialized = rdflib_bug_rx.sub('', serialized)
                with open(out_path, 'w') as fp:
                    fp.write(serialized)

            if int(f[:-4]) % 10_000 == 0:
                print(f[:-4])

0000000000


KeyboardInterrupt: 